In [15]:
# load the source code reload module
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
! python3 --version

Python 3.10.12


In [3]:
import os
import gc

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [16]:
import json
from xmlschema import XMLSchema

os.chdir( "/var/model/genie-in-the-box/src" )

import lib.utils.util     as du
import lib.utils.util_xml as dux
import lib.utils.util_pytorch as dupt

from lib.agents.agent         import XXX_Agent

from ephemera.prompts.xml_fine_tuning_prompt_generator import XmlFineTuningPromptGenerator


## Get Validation dataset

In [5]:
# path = "/var/model/genie-in-the-box/src/ephemera/prompts/data/voice-commands-xml-validate.jsonl"
# deepily_dataset_validation = du.get_file_as_list( path )
# deepily_dataset_validation = [ json.loads( line ) for line in deepily_dataset_validation ]
# len( deepily_dataset_validation )

In [6]:
# def generate_text( foo_tokenizer, model, question, max_new_tokens=256, debug=False ):
#     
#     instruction = f"""### Instruction:
#     Use the Task below and the Input given to write a Response that can solve the following Task:
# 
#     ### Task:
#     Your job is to discern the intent of a human voice command transcription and translate it into a standardized command that a browser on your computer would understand.
# 
#     You will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `'search new tab', 'search current tab', 'search google new tab', 'search google current tab', 'search google scholar new tab', 'search google scholar current tab' and 'none'`.
# 
#     Requirement: You MUST NOT use python code to answer this question.
#     Requirement: You MUST use your linguistic knowledge and intuition to answer this question.
#     Hint: Anything that isn't a part of the command itself should be treated as arguments related to the command.
# 
#     ### Input:
#     
#     Below is the raw human voice command transcription formatted using simple XML:
#     
#     <human>
#         <voice-command>{question}</voice-command>
#     </human>
#     
#     The standardized command that you translate MUST be returned wrapped in simple, well-formed XML:
#     
#     <response>
#         <browser-command></browser-command>
#         <args></args>
#     </response>
# 
#     Requirement: The first word of your response MUST be `<response>`
# 
#     ### Response:"""
#     
#     
#     device = "cuda:0"
#     inputs = foo_tokenizer( instruction, return_tensors="pt" ).to( device )
#     
#     stop_token_id = foo_tokenizer.encode( "</response>" )[ 0 ]
#     
#     generation_output = model.generate(
#         input_ids=inputs[ "input_ids" ],
#         attention_mask=inputs[ "attention_mask" ],
#         max_new_tokens=max_new_tokens,
#         eos_token_id=stop_token_id,
#         pad_token_id=stop_token_id
#     )
#         
#     if debug: 
#         print( "generation_output[ 0 ]:", generation_output[ 0 ], end="\n\n" )
#         print( "generation_output[ 0 ].shape:", generation_output[ 0 ].shape, end="\n\n" )
#     
#     # Skip decoding the prompt part of the output   
#     input_length = inputs[ "input_ids" ].size( 1 )
#     raw_output = foo_tokenizer.decode( generation_output[ 0 ][ input_length: ] )
#     
#     if debug: 
#         print( "raw_output:", raw_output, end="\n\n" )
#         print(  "len( raw_output ):", len( raw_output ), end="\n\n")
#     
#     # response   = raw_output.split( "### Response:" )[ 1 ]
#     
#     response = raw_output.replace( "</s><s>", "" ).strip()
#     
#     return response
# 
# question = "Ask Google scholar about QLORA and PEFT fine-tuning for XML output, show results in Another tab"
# 
# # for line in generate_text( tokenizer, base_model, question ).split( "\n" ): print( line )

In [7]:
os.chdir( "/var/model/Phind-CodeLlama-34B-v2" )

In [6]:
def get_base_model_and_tokenizer( model_path=".", tokenizer_path=".", use_quant_config=True, device_map="auto" ):
    
    compute_dtype = getattr( torch, "float16" )
    
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype
    )
    print( bnb_config )
    tokenizer              = AutoTokenizer.from_pretrained( tokenizer_path )
    tokenizer.pad_token    = tokenizer.eos_token
    tokenizer.padding_side = "right"
    
    if use_quant_config:
        # ¡OJO! Why were we turning off the cash here? It makes a big performance difference: 21 vs 14 tokens per second
        base_model = AutoModelForCausalLM.from_pretrained(
            model_path, quantization_config=bnb_config, device_map=device_map, low_cpu_mem_usage=True, use_cache=True, attn_implementation="flash_attention_2"
        )
    else:
        base_model = AutoModelForCausalLM.from_pretrained(
            model_path, device_map=device_map, low_cpu_mem_usage=True, use_cache=True, attn_implementation="flash_attention_2"
        )
    
    return base_model, tokenizer
    
# base_model, tokenizer = get_base_model_and_tokenizer( device_map="cuda:1" )


In [15]:
dupt.print_device_allocation( base_model )

model.embed_tokens.weight: cuda:0
model.layers.0.self_attn.q_proj.weight: cuda:0
model.layers.0.self_attn.k_proj.weight: cuda:0
model.layers.0.self_attn.v_proj.weight: cuda:0
model.layers.0.self_attn.o_proj.weight: cuda:0
model.layers.0.mlp.gate_proj.weight: cuda:0
model.layers.0.mlp.up_proj.weight: cuda:0
model.layers.0.mlp.down_proj.weight: cuda:0
model.layers.0.input_layernorm.weight: cuda:0
model.layers.0.post_attention_layernorm.weight: cuda:0
model.layers.1.self_attn.q_proj.weight: cuda:0
model.layers.1.self_attn.k_proj.weight: cuda:0
model.layers.1.self_attn.v_proj.weight: cuda:0
model.layers.1.self_attn.o_proj.weight: cuda:0
model.layers.1.mlp.gate_proj.weight: cuda:0
model.layers.1.mlp.up_proj.weight: cuda:0
model.layers.1.mlp.down_proj.weight: cuda:0
model.layers.1.input_layernorm.weight: cuda:0
model.layers.1.post_attention_layernorm.weight: cuda:0
model.layers.2.self_attn.q_proj.weight: cuda:0
model.layers.2.self_attn.k_proj.weight: cuda:0
model.layers.2.self_attn.v_proj.we

In [15]:
dupt.is_allocated_to_cpu( base_model )

False

In [16]:
modules = base_model.named_modules()
for idx, module in enumerate( modules ):
    print( f"idx [{idx}] name: [{module[ 0 ]}] type: [{module[ 1 ]}]" )

idx [0] name: [] type: [LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 8192)
    (layers): ModuleList(
      (0-47): 48 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): Linear4bit(in_features=8192, out_features=8192, bias=False)
          (k_proj): Linear4bit(in_features=8192, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=8192, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=8192, out_features=8192, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=8192, out_features=22016, bias=False)
          (up_proj): Linear4bit(in_features=8192, out_features=22016, bias=False)
          (down_proj): Linear4bit(in_features=22016, out_features=8192, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
   

In [9]:
from peft import PeftModel

def load_adapter( model, adapter_path, use_flash_attention_2=True ):
    
    return PeftModel.from_pretrained( model, adapter_path, use_flash_attention_2=use_flash_attention_2 )

adapter_plus_model = load_adapter( base_model, "adapters/01-browser-vox-command", use_flash_attention_2=True )

In [15]:
adapter_plus_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 8192)
        (layers): ModuleList(
          (0-47): 48 x LlamaDecoderLayer(
            (self_attn): LlamaFlashAttention2(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=8192, out_features=8192, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=8192, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=8192, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Linear

In [11]:
%autoreload
dupt.print_device_allocation( adapter_plus_model )

base_model.model.model.embed_tokens.weight: cuda:1
base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight: cuda:1
base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight: cuda:1
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight: cuda:1
base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight: cuda:1
base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight: cuda:1
base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight: cuda:1
base_model.model.model.layers.0.self_attn.v_proj.base_layer.weight: cuda:1
base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight: cuda:1
base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight: cuda:1
base_model.model.model.layers.0.self_attn.o_proj.base_layer.weight: cuda:1
base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight: cuda:1
base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight: cuda:1
base_model.model.

In [12]:
%autoreload
dupt.is_allocated_to_cpu( adapter_plus_model )

False

## Naïvely merge model and adapter

In [13]:
adapter_plus_model = adapter_plus_model.merge_and_unload()
adapter_plus_model.save_pretrained( "./merged-naive/", safe_serialization=True )

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:257: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [14]:
import transformers
transformers.__version__

'4.37.0.dev0'

## Dequantize and then merge  

In [12]:
import torch
import peft
import json
import shutil
from peft.utils import _get_submodules
import os
import bitsandbytes as bnb
from bitsandbytes.functional import dequantize_4bit
from peft import PeftModel
from transformers import AutoModelForCausalLM, LlamaForCausalLM, LlamaTokenizer, BitsAndBytesConfig
import gc
import copy


def dequantize_model( model, tokenizer, to='./dequantized-model', dtype=torch.float16, device="cpu" ):
    """
    'model': the peftmodel you loaded with qlora.
    'tokenizer': the model's corresponding hf's tokenizer.
    'to': directory to save the dequantized model
    'dtype': dtype that the model was trained using
    'device': device to load the model to
    """
    
    os.makedirs( to, exist_ok=True )
    
    cls = bnb.nn.Linear4bit
    
    with torch.no_grad():
        for name, module in model.named_modules():
            if isinstance( module, cls ):
                print( f"Dequantizing `{name}`..." )
                quant_state = copy.deepcopy( module.weight.quant_state )
                print( f"quant_state.__dict__: [{quant_state.__dict__}]" )
                try:
                    quant_state.dtype = dtype
                    print( f"SUCCESS setting quant_state.dtype [{quant_state}] to dtype: [{dtype}]" )
                except Exception as e:
                    print( f"FAILed   to set quant_state.dtype [{quant_state}] to dtype: [{dtype}] [{e}]" )
                    # quant_state = None
                
                weights = dequantize_4bit( module.weight.data, quant_state=quant_state, quant_type="nf4" ).to( dtype )
                
                new_module = torch.nn.Linear( module.in_features, module.out_features, bias=None, dtype=dtype )
                new_module.weight = torch.nn.Parameter( weights )
                new_module.to( device=device, dtype=dtype )
                
                parent, target, target_name = _get_submodules( model, name )
                setattr( parent, target_name, new_module )
                
            else:
                print( f"Skipping `{name}`..." )
        
        # a hack, setting this to avoid hf's saving error because hf
        # itself does not support saving a model that is registered to be loaded in 4bit.
        model.is_loaded_in_4bit = False
        
        print( "Saving dequantized model & tokenizer..." )
        model.save_pretrained( to )
        tokenizer.save_pretrained( to )
        config_data = json.loads( open( os.path.join( to, 'config.json' ), 'r' ).read() )
        config_data.pop( "quantization_config", None )
        config_data.pop( "pretraining_tp", None )
        with open( os.path.join( to, 'config.json' ), 'w' ) as config:
            config.write( json.dumps( config_data, indent=2 ) )
        
        return model


#To which precision do you want to dequantize?
dtype = torch.bfloat16

# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=dtype,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
# )

model_name = "."
try:
    print( f"Dequantizing model {model_name}.." )
    
    # model = LlamaForCausalLM.from_pretrained(
    #     model_name,
    #     quantization_config=quantization_config,
    #     device_map={ "": 0 }
    # )
    du.print_banner( "base_model: PRE dequantize" )
    print( base_model )
    base_model = dequantize_model( base_model, tokenizer, to='./dequantized-base-model/', dtype=dtype )
    du.print_banner( "base_model: POST dequantize" )
    print( base_model )
    
    # base_model_plus_adapter = AutoModelForCausalLM.from_pretrained( base_model, "adapters/00-browser-vox-command" )
    # du.print_banner( "PRE merge base_model_plus_adapter" )
    # print( base_model_plus_adapter )
    # 
    # base_model_plus_adapter = base_model_plus_adapter.merge_and_unload()
    # du.print_banner( "POST merged base_model_plus_adapter" )
    # print( base_model_plus_adapter )
    # 
    # base_model_plus_adapter.save_pretrained( "./merged/", safe_serialization=True )
    # tokenizer.save_pretrained( "./merged/" )
    
except Exception as e:
    
    print( f"An error occurred: {e}" )
    # Print Stacktrace
    import traceback
    traceback.print_exc()
    
    # # Delete the model object if it exists
    # if 'model' in locals():
    #     del base_model
    # 
    # # Clear the GPU cache
    # torch.cuda.empty_cache()
    # 
    # # Run the garbage collection
    # gc.collect()
    # 
    # print( "Model, GPU cache, and garbage have been cleared." )


Dequantizing model ...
------------------------------------------------------------------------------------------------------------------------
- base_model: PRE dequantize
------------------------------------------------------------------------------------------------------------------------

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 8192)
    (layers): ModuleList(
      (0-47): 48 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): Linear4bit(in_features=8192, out_features=8192, bias=False)
          (k_proj): Linear4bit(in_features=8192, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=8192, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=8192, out_features=8192, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=8192, out_features=22016, bias=False)
          (up_proj): Linear4

In [13]:
base_model_plus_adapter = AutoModelForCausalLM.from_pretrained( base_model, "adapters/00-browser-vox-command" )
du.print_banner( "PRE merge base_model_plus_adapter" )
print( base_model_plus_adapter )

# base_model_plus_adapter = base_model_plus_adapter.merge_and_unload()
# du.print_banner( "POST merged base_model_plus_adapter" )
# print( base_model_plus_adapter )
# 
# base_model_plus_adapter.save_pretrained( "./merged/", safe_serialization=True )
# tokenizer.save_pretrained( "./merged/" )

------------------------------------------------------------------------------------------------------------------------
- PRE merge base_model_plus_adapter
------------------------------------------------------------------------------------------------------------------------

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 8192)
        (layers): ModuleList(
          (0-47): 48 x LlamaDecoderLayer(
            (self_attn): LlamaFlashAttention2(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=8192, out_features=8192, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=8192, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default)

In [14]:
# base_model_plus_adapter = AutoModelForCausalLM.from_pretrained( base_model, "adapters/00-browser-vox-command" )
# du.print_banner( "PRE merge base_model_plus_adapter" )
# print( base_model_plus_adapter )

base_model_plus_adapter = base_model_plus_adapter.merge_and_unload()
du.print_banner( "POST merged base_model_plus_adapter" )
print( base_model_plus_adapter )
# 
# base_model_plus_adapter.save_pretrained( "./merged/", safe_serialization=True )
# tokenizer.save_pretrained( "./merged/" )

------------------------------------------------------------------------------------------------------------------------
- POST merged base_model_plus_adapter
------------------------------------------------------------------------------------------------------------------------

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 8192)
    (layers): ModuleList(
      (0-47): 48 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): Linear(in_features=8192, out_features=8192, bias=False)
          (k_proj): Linear(in_features=8192, out_features=1024, bias=False)
          (v_proj): Linear(in_features=8192, out_features=1024, bias=False)
          (o_proj): Linear(in_features=8192, out_features=8192, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=8192, out_features=22016, bias=False)
          (up_proj): Linear(in_features=8192, out_features=220

In [15]:
base_model_plus_adapter.save_pretrained( "./merged/", safe_serialization=True )
tokenizer.save_pretrained( "./merged/" )

('./merged/tokenizer_config.json',
 './merged/special_tokens_map.json',
 './merged/tokenizer.model',
 './merged/added_tokens.json',
 './merged/tokenizer.json')

## load merged model

In [9]:
os.chdir( "/var/model/Phind-CodeLlama-34B-v2" )
os.getcwd()

'/var/model/Phind-CodeLlama-34B-v2'

In [10]:
# device_map = "cuda:1"
device_map = "auto"
merged_model, tokenizer = get_base_model_and_tokenizer( "./merged-naive/", ".", use_quant_config=False, device_map=device_map )

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


BitsAndBytesConfig {
  "bnb_4bit_compute_dtype": "float16",
  "bnb_4bit_quant_type": "nf4",
  "bnb_4bit_use_double_quant": true,
  "llm_int8_enable_fp32_cpu_offload": false,
  "llm_int8_has_fp16_weight": false,
  "llm_int8_skip_modules": null,
  "llm_int8_threshold": 6.0,
  "load_in_4bit": true,
  "load_in_8bit": false,
  "quant_method": "bitsandbytes"
}


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [21]:
import gc

def free_gpu( models ):
    
    for model in models:
        del model
        model = None
    
    torch.cuda.empty_cache()
    gc.collect()
    
free_gpu( [ base_model, adapter_plus_model ] )

In [12]:
import pandas as pd

In [13]:
validation_df = pd.read_json( "/var/model/genie-in-the-box/src/ephemera/prompts/data/voice-commands-xml-validate.jsonl", lines=True ).sample( 100, random_state=42 )
validation_df.head()

,instruction,input,output,prompt,gpt_message
6252,Your job is to discern the intent of a human v...,\n Below is the raw human voice command...,\n <response>\n <browser-com...,### Instruction:\n Use the Task and Input g...,"{'messages': [{'role': 'system', 'content': 'I..."
4684,Your job is to discern the intent of a human v...,\n Below is the raw human voice command...,\n <response>\n <browser-com...,### Instruction:\n Use the Task and Input g...,"{'messages': [{'role': 'system', 'content': 'I..."
1731,Your job is to discern the intent of a human v...,\n Below is the raw human voice command...,\n <response>\n <browser-com...,### Instruction:\n Use the Task and Input g...,"{'messages': [{'role': 'system', 'content': 'I..."
4742,Your job is to discern the intent of a human v...,\n Below is the raw human voice command...,\n <response>\n <browser-com...,### Instruction:\n Use the Task and Input g...,"{'messages': [{'role': 'system', 'content': 'I..."
4521,Your job is to discern the intent of a human v...,\n Below is the raw human voice command...,\n <response>\n <browser-com...,### Instruction:\n Use the Task and Input g...,"{'messages': [{'role': 'system', 'content': 'I..."


In [14]:
validation_df.shape

(100, 5)

In [18]:
%autoreload

xml_ftp_generator = XmlFineTuningPromptGenerator( path_prefix="/var/model/genie-in-the-box", debug=True, verbose=False )

validation_df = xml_ftp_generator.generate_responses( validation_df, tokenizer=tokenizer, model=merged_model, switch="huggingface", model_name=XXX_Agent.PHIND_34B_v2 )
validation_df = xml_ftp_generator.validate_responses( validation_df )

Commands file for [search new tab] exists: True
Commands file for [search current tab] exists: True
Commands file for [search google new tab] exists: True
Commands file for [search google current tab] exists: True
Commands file for [search google scholar new tab] exists: True
Commands file for [search google scholar current tab] exists: True

Using HuggingFace model_name [TGI/Phind-CodeLlama-34B-v2] in memory...

Processing call [001] out of [100] = [1.0%]... 
Asking LLM [TGI/Phind-CodeLlama-34B-v2]...
Asking LLM [TGI/Phind-CodeLlama-34B-v2]... Done! in 2,929 ms
Tokens per second [48.1]
Response: [<response><browser-command>search google scholar new tab</browser-command><args>URLError</args></response>]

Processing call [002] out of [100] = [2.0%]... 
Asking LLM [TGI/Phind-CodeLlama-34B-v2]...
Asking LLM [TGI/Phind-CodeLlama-34B-v2]... Done! in 3,186 ms
Tokens per second [64.3]
Response: [<response><browser-command>search google scholar new tab</browser-command><args>How can system exi

In [19]:
xml_ftp_generator.print_validation_stats( validation_df )


------------------------------------------------------------------------------------------------------------------------
- Validation Stats
------------------------------------------------------------------------------------------------------------------------

               Is valid xml 100.0%
          Contains response 100.0%
   Contains browser command 100.0%
              Contains args 100.0%
          Response is exact 47.0%
Response has correct values 47.0%
 Browser command is correct 60.0%
            Args is correct 81.0%


In [27]:
os.chdir( "/var/model/genie-in-the-box/src" )
os.getcwd()

'/var/model/genie-in-the-box/src'

In [28]:
%autoreload
from ephemera.prompts.xml_fine_tuning_prompt_generator import XmlFineTuningPromptGenerator

# # Advice from phind when auto reload fails: https://www.phind.com/search?cache=l7c51x1v3tids6l43mgkkm9j
# os.chdir( "/var/model/genie-in-the-box/src" )
# %run "ephemera/prompts/xml_fine_tuning_prompt_generator.py"  

xml_ftp_generator = XmlFineTuningPromptGenerator( path_prefix="/var/model/genie-in-the-box" )

validation_df = xml_ftp_generator.validate_prompts_and_responses( validation_df )

xml_ftp_generator.print_validation_stats( validation_df )

#  Results from a 1000 sample run:
# ------------------------------------------------------------------------------------------------------------------------
# - Validation Stats: After training adapter --or-- After loading pre-trained adapter and adapter
# ------------------------------------------------------------------------------------------------------------------------
# 
#                Is valid xml 100.0%
#           Contains response 100.0%
#    Contains browser command 100.0%
#               Contains args 100.0%
#           Response is exact 99.3%
# Response has correct values 99.3%
#  Browser command is correct 99.6%
#             Args is correct 99.7%

# ------------------------------------------------------------------------------------------------------------------------
# - Validation Stats: After naïvely merging adapter and model
# ------------------------------------------------------------------------------------------------------------------------
# 
#                Is valid xml 100.0%
#           Contains response 100.0%
#    Contains browser command 100.0%
#               Contains args 100.0%
#           Response is exact 54.0%
# Response has correct values 54.0%
#  Browser command is correct 61.0%
#             Args is correct 83.0%
# 
# ------------------------------------------------------------------------------------------------------------------------
# - Validation Stats: Baseline, without adapter or training
# ------------------------------------------------------------------------------------------------------------------------
# 
#                Is valid xml 100.0%
#           Contains response 100.0%
#    Contains browser command 100.0%
#               Contains args 100.0%
#           Response is exact 46.0%
# Response has correct values 46.0%
#  Browser command is correct 54.0%
#             Args is correct 78.0%

# ------------------------------------------------------------------------------------------------------------------------
# - Validation Stats: After dequantizing and merging adapter and model
# ------------------------------------------------------------------------------------------------------------------------
# 
#                Is valid xml 100.0%
#           Contains response 100.0%
#    Contains browser command 100.0%
#               Contains args 100.0%
#           Response is exact 41.0%
# Response has correct values 41.0%
#  Browser command is correct 48.0%
#             Args is correct 78.0%

Commands file for [search new tab] exists: True
Commands file for [search current tab] exists: True
Commands file for [search google new tab] exists: True
Commands file for [search google current tab] exists: True
Commands file for [search google scholar new tab] exists: True
Commands file for [search google scholar current tab] exists: True

------------------------------------------------------------------------------------------------------------------------
- Validation Stats
------------------------------------------------------------------------------------------------------------------------

               Is valid xml 100.0%
          Contains response 100.0%
   Contains browser command 100.0%
              Contains args 100.0%
          Response is exact 41.0%
Response has correct values 41.0%
 Browser command is correct 48.0%
            Args is correct 78.0%
